## Concatenar y append de datasets

In [2]:
import pandas as pd

In [3]:
#dataset con caracteristicas de tipos de vino tinto
red_wine = pd.read_csv("../datasets/wine/winequality-red.csv",sep=";")

In [ ]:
red_wine.head()

In [ ]:
red_wine.columns.values #investigamos las columnas del dataset

In [ ]:
red_wine.shape

In [4]:
#dataset con caracterirticas de tipos de vino blanco
white_wine = pd.read_csv("../datasets/wine/winequality-white.csv",sep=";")

In [ ]:
white_wine.head()

In [ ]:
white_wine.shape


### Ambos dataset tienen las mismas columnas y los mismos tipos de datos, ejemplo ideal para unirlos uno debajo del otro
En python tenemos dos tipos de ejes,
* axis = 0 denota el eje horizontal
* axis =1 denota el eje vertical

In [ ]:
# concatenamos el dataset del vino tinto ENCIMA del dataset del vino blanco en el eje horizontal (apilar datasets)
wine_data = pd.concat([red_wine,white_wine],axis = 0)

In [ ]:
wine_data.shape


In [ ]:
wine_data.head()# el data set del vino tinto quedo encima

In [ ]:
data1 = wine_data.head(10)
data2 = wine_data[300:310]
data3 = wine_data.tail(10)

In [ ]:
wine_scramble = pd.concat([data1,data2,data3],axis =0)

In [ ]:
#verificar como se apilan los datasets en el orden que especificamos en concat
wine_scramble

## Concatenar Datos Distribudios: Cientos de Datasets

In [5]:
#datos diarios de mediciones de calidad del aire desde multiples sensores
data = pd.read_csv("../datasets/distributed-data/001.csv")


In [ ]:
data.head()

In [ ]:
data.shape

* Importar el primer fichero
* Bucle para ir recorriendo los archivos 
    * deben tener un nombre consistente para que se pueda iterar a traves de ellos
    * importar los ficheros de a uno
    * Cada uno se va agregando al final del primer fichero ya cargado
* Repetir el bucle hasta cargarlos todos

In [6]:
#empezamos cargando el primer datasets, ver como el nombre es un numero secuencial iterable
filepath = "../datasets/distributed-data/"
data = pd.read_csv("../datasets/distributed-data/001.csv")
final_lenght = len(data)
#se inicia el bucle
for i in range(1,333):
    if i <10:
        filename = "00"+str(i)
    if 10 <= i <100:
        filename = "0"+str(i)
    if i>=100:
        filename = str(i)
    file = filepath + filename +".csv"
    temp_data = pd.read_csv(file)
    final_lenght += len(temp_data)
    
    data = pd.concat([data, temp_data], axis= 0)

In [ ]:
data.shape

In [ ]:
data.tail()


In [ ]:
data.head()


In [ ]:
final_lenght == data.shape[0]

### Joins de datasets

In [7]:
filepath ="../datasets/athletes/"

In [8]:
data_main = pd.read_csv(filepath+ "Medals.csv",encoding ="ISO-8859-1")

In [ ]:
data_main.head()


In [18]:

a = data_main["Athlete"].unique().tolist()## cuantos atletas aparecen una sola vez en el dataset
len(a)

6956

In [9]:
#Cargamos otro dataset con info adicional de los atletas, en este caso el pais de origen
data_country = pd.read_csv(filepath + "Athelete_Country_Map.csv",encoding = "ISO-8859-1")

In [ ]:
data_country.head()

In [ ]:
#es distinto, puede ser que atletas hayan ganado medallas por a distintos paises, o sus paises de origen cambiaron de nombre
len(data_country)

In [ ]:
data_country[data_country["Athlete"] == "Aleksandar Ciric"]

In [10]:
# data sets Atletas segun su deporte
data_sports = pd.read_csv(filepath + "Athelete_Sports_Map.csv",encoding="ISO-8859-1")

In [ ]:
#tambien el numero de deportistas no coincide con los otros datasets, un mismo atleta puede participar en varios deportes
data_sports.head()

In [11]:
data_sports[(data_sports["Athlete"]=="Chen Jing") |
            (data_sports["Athlete"]=="Richard Thompson") |
            (data_sports["Athlete"]=="Matt Ryan")]

,Athlete,Sport
528,Richard Thompson,Athletics
1308,Chen Jing,Volleyball
1419,Chen Jing,Table Tennis
2727,Matt Ryan,Rowing
5003,Matt Ryan,Equestrian
5691,Richard Thompson,Baseball


In [12]:
#union de datasets de atletas, yel de atletas y sus paises de origen, se debe especificar el orden y,
#tambien la columna para unir que se usa como clave para el join, es un tipo de inner join
data_main_country = pd.merge(left = data_main, right =data_country,
                             left_on="Athlete", right_on ="Athlete")

In [13]:
data_main_country.head()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
0,Michael Phelps,23.0,2008,08/24/2008,8,0,0,8,United States
1,Michael Phelps,19.0,2004,08/29/2004,6,0,2,8,United States
2,Michael Phelps,27.0,2012,08/12/2012,4,2,0,6,United States
3,Natalie Coughlin,25.0,2008,08/24/2008,1,2,3,6,United States
4,Natalie Coughlin,21.0,2004,08/29/2004,2,2,1,5,United States


In [14]:
data_main_country.shape


(8657, 9)

In [ ]:
#se duplican algunos datos al estar presentes dos veces en ambos dataset
data_main_country[data_main_country["Athlete"]=="Aleksandar Ciric"]

In [15]:
#eliminar filas duplicadas del dataset principal, dejara solo un pais por atleta
data_country_dp = data_country.drop_duplicates(subset="Athlete")

In [19]:
#verificamos que ahora el dataset de paises es igual al numero total de atletas sin duplicados
len(data_country_dp) == len(a)

True

In [22]:
#aplicamos el merge ahora con el dataset de paises sin duplicados
data_main_country = pd.merge(left = data_main, right =data_country_dp,
                             left_on="Athlete", right_on ="Athlete")

In [23]:
data_main_country.shape

(8618, 9)

In [24]:
#seguimos haciendo merge, los deportes por atleta unir al dataset principal de atletas, quitando los duplicados
data_sport_dp = data_sports.drop_duplicates(subset="Athlete")

In [29]:
#verificamos que coincide el numero de atletas con el de deportes, ya sin duplicados
len(data_sport_dp) ==len(a)

True

In [26]:
data_final = pd.merge(left=data_main_country,right=data_sport_dp,
                     left_on="Athlete", right_on="Athlete")

In [28]:
len(data_final)

8618

In [32]:
data_final.head()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country,Sport
0,Michael Phelps,23.0,2008,08/24/2008,8,0,0,8,United States,Swimming
1,Michael Phelps,19.0,2004,08/29/2004,6,0,2,8,United States,Swimming
2,Michael Phelps,27.0,2012,08/12/2012,4,2,0,6,United States,Swimming
3,Natalie Coughlin,25.0,2008,08/24/2008,1,2,3,6,United States,Swimming
4,Natalie Coughlin,21.0,2004,08/29/2004,2,2,1,5,United States,Swimming
